# 1. Description
**Project ini disusun untuk mengambil data dari halaman sebuah website, kemudian diolah dan disimpan ke dalam database PostgreSQL.**

**Website:**
https://www.dekoruma.com/search?query=meja

# 2. Import Libraries
Bagian ini berisikan proses mengimport semua library yang dibutuhkan pada project ini.

In [166]:
# Mengimport pandas (untuk manipulasi data)
import pandas as pd

# Mengimport numpy
import numpy as np

# Mengimport BeautifulSoup dan request untuk web scraping (menggunakan HTML)
from bs4 import BeautifulSoup
import requests

# Mengimport selenium untuk otomatisasi website menggunakan chrome
from selenium import webdriver

# Mengimport time untuk mengatur waktu (menetapkan jeda dll)
import time

# 3. Extract Data
Bagian ini berisikan proses untuk mengambil data yang akan digunakan untuk analisis lebih lanjut. Pada proses ini, data diambil dari website e-commerce yang menjual barang perabotan rumah tangga yaitu 'Dekoruma' menggunakan web scraping. 

In [126]:
# Melakukan scraping data dari website

driver = webdriver.Chrome() # Memanggil driver
nama_produk = [] # Membuat variabel (list) nama_produk (kosong) untuk menyimpan data nama produk
harga_sebelum_diskon = [] # Membuat variabel (list) harga_sebelum_diskon (kosong) untuk menyimpan data harga normal produk (tanpa diskon)
diskon = [] # Membuat variabel (list) diskon (kosong) untuk menyimpan data diskon pada masing-masing produk
harga_setelah_diskon = [] # Membuat variabel (list) harga_setelah_diskon (kosong) untuk menyimpan data harga produk setelah dipotong diskon
terjual_atau_ulasan = [] # Membuat variabel (list) terjual_atau_ulasan (kosong) untuk menyimpan data keterangan yang tersedia terkait produk (dapat berupa jumlah yang terjual atau ulasan produk)
gratis_ongkir = [] # Membuat variabel (list) gratis_ongkir (kosong) untuk menyimpan data jenis gratis ongkir apa yang akan diperoleh jika membeli produk
for page in range(1,6): # Range dibuat sampai dengan 6 untuk men-scraping data sampai page 5
    dekoruma = (f'https://www.dekoruma.com/search?query=meja&page={page}') # Menyimpan link website dalam variabel
    driver.get(dekoruma)
    time.sleep(10) # Untuk memberikan jeda agar halaman bisa dimuat penuh sebelum dilakukan scraping
    scroll_pause_time = 10 # Untuk memberi jeda di ketika melakukan scroll 
    last_height = driver.execute_script("return document.body.scrollHeight") # Untuk mendapatkan/mendefinisikan tinggi halaman awal (akan digunakan untuk membuat scrolling otomatis)
    while True: # Loop untuk melakukan scrolling ke bawah
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight);") # Scroll ke bawah sampai bagian terbawah halaman
        time.sleep(5) # Jeda 5 detik agar elemen baru termuat
        new_height = driver.execute_script("return document.body.scrollHeight") # Mengambil (mendefinisikan) tinggi halaman setelah scroll
        if new_height == last_height:
            break  # Jika tinggi tidak berubah, berhenti scrolling
        last_height = new_height # Perbarui tinggi halaman sebelum iterasi berikutnya

    time.sleep(5)  # Jeda setelah scrolling sebelum mulai parsing HTML
    time.sleep(5)  # Untuk memberikan jeda agar halaman bisa dimuat penuh setelah di scroll kebawah dan sebelum berpindah ke halaman selanjutnya
    html = driver.page_source # Menyimpan HTML halaman
    soup = BeautifulSoup(html, 'html.parser') # Parsing HTML menggunakan BeautifulSoup
    for data in soup.find_all('div', {'class', 'css-1dbjc4n r-z2wwpe r-27sa8p r-13awgt0 r-1pi2tsx r-1wtj0ep r-1jkjb r-1kb76zh r-bgnin r-1udh08x'}): # Mencari semua elemen <div> yang berisi informasi produk
        try:
            nama_produk.append(data.find('div', {'class':'css-901oao css-cens5h r-1umadx r-1enofrn r-8ay8q1 r-1cwl3u0 r-15zivkp'}).get_text()) # Mengambil teks nama produk dan menambahkannya ke list
        except:
            nama_produk.append(None)  # Jika gagal menemukan elemen, tambahkan None ke list
        try:
            harga_sebelum_diskon.append(data.find('div', {'class':'css-901oao r-1umadx r-1enofrn r-8ay8q1 r-1cwl3u0 r-142tt33'}).get_text()) # Mengambil teks harga sebelum diskon dan menambahkannya ke list
        except:
            try:
                harga_sebelum_diskon.append(data.find('div', {'class': 'css-901oao r-1umadx r-a023e6 r-b88u0q r-rjixqe'}).get_text()) # Jika tidak ada harga sebelum diskon pada letak sebelumnya, ambil dari tempat ini. (Ini perlu dilakukan karena dalam website terdapat produk yang tidak memiliki diskon. Produk-produk yang tidak memiliki diskon tersebut memiliki letak harga yang berbeda dengan yang memiliki diskon)
            except:
                harga_sebelum_diskon.append(None)   # Jika gagal menemukan elemen, tambahkan None ke list
        try:
            diskon.append(data.find('div', {'class':'css-901oao r-1umadx r-10x49cs r-b88u0q r-3ucsff r-56xrmm'}).get_text())
        except:
            diskon.append(None) # Jika gagal menemukan elemen, tambahkan None ke list
        try:
            harga_setelah_diskon.append(data.find('div', {'class':'css-901oao r-1umadx r-a023e6 r-b88u0q r-rjixqe'}).get_text()) # Mengambil teks harga setelah diskon dan menambahkannya ke list
        except:
            try:
                harga_setelah_diskon.append(data.find('div', {'class': 'css-1dbjc4n r-1ecahn0 r-lchren r-14gqq1x r-iphfwy r-1m04atk r-1pyaxff r-1h8ys4a'}).get_text()) # Jika tidak ada harga setelah diskon pada letak sebelumnya, ambil dari tempat ini. (Ini perlu dilakukan karena dalam website terdapat beberapa produk yang memiliki diskon, letal harga setelah diskonnya berbeda dari posisi sebelumnya)
            except:
                harga_setelah_diskon.append(None)  # Jika gagal menemukan elemen, tambahkan None ke list
        try:
            terjual_atau_ulasan.append(data.find('div', {'class':'css-901oao r-1umadx r-1gkfh8e r-8ay8q1 r-1f529hi'}).get_text()) # Mengambil teks terjual atau ulasan dan menambahkannya ke list (posisi ulasan dan terjual berada di tempat yang sama)
        except:
            terjual_atau_ulasan.append(None) # Jika gagal menemukan elemen, tambahkan None ke list
        try:
            gratis_ongkir.append(data.find('div', {'class':'css-901oao r-1umadx r-1enofrn r-b88u0q r-8ay8q1 r-1cwl3u0'}).get_text()) # Mengambil gratis ongkir yang didapatkan dan menambahkannya ke list 
        except:
            gratis_ongkir.append(None) 

In [127]:
# Melihat apakah jumlah data di setiap variabel sudah sama jumlahnya

print(len(nama_produk))
print(len(harga_sebelum_diskon))
print(len(diskon))
print(len(harga_setelah_diskon))
print(len(terjual_atau_ulasan))
print(len(gratis_ongkir))

120
120
120
120
120
120


Dari output di atas diketahui bahwa seluruh variabel (kolom) memiliki panjang data yang sama.

In [184]:
# Membuat dataframe

df = pd.DataFrame() 
df['Nama Produk'] = nama_produk
df['Harga Sebelum Diskon'] = harga_sebelum_diskon
df['Diskon'] = diskon
df['Harga Setelah Diskon'] = harga_setelah_diskon
df['Jumlah Terjual atau Ulasan'] = terjual_atau_ulasan
df['Gratis Ongkir'] = gratis_ongkir

In [187]:
# Menutup driver

driver.close()

In [188]:
# Memeriksa dataframe

df

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,"Rp1,485,000",42%,"Rp859,000",(4055 Terjual),se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,"Rp1,980,000",47%,"Rp1,049,000",(3612 Terjual),se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,"Rp2,178,000",52%,"Rp1,029,000",(2661 Terjual),se-Jabodetabek
3,Heim Studio ISY Meja TV,"Rp1,434,000",39%,"Rp869,000",(3153 Terjual),se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,"Rp3,636,000",48%,"Rp1,859,000",(3135 Terjual),se-Jabodetabek
...,...,...,...,...,...,...
115,Tenzo Living ALANO Meja Tamu,"Rp8,670,000",25%,"Rp6,499,000",None,se-Jabodetabek
116,Tenzo Living VERLIN Meja Tamu,"Rp10,000,000",25%,"Rp7,499,000",None,se-Jabodetabek
117,Heim Studio SOKA Meja Makan,"Rp4,053,000",42%,"Rp2,349,000",(5 Ulasan),se-Jabodetabek
118,Cubic Cubic Meja Komputer Rumah - Meja Kerja L...,"Rp430,000",50%,"Rp215,000",(34 Terjual),se-Jawa


Data telah berhasil di-scraping dan dimasukkan ke dalam dataframe df, yang selanjutnya dapat digunakan untuk analisis lebih lanjut.

# 4. Data Pre-Processing

## 4.1. Exploration Data Analysis (Before Cleaning Process)

In [189]:
# Menampilkan informasi atau rangkuman dari dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Nama Produk                 120 non-null    object
 1   Harga Sebelum Diskon        120 non-null    object
 2   Diskon                      114 non-null    object
 3   Harga Setelah Diskon        120 non-null    object
 4   Jumlah Terjual atau Ulasan  101 non-null    object
 5   Gratis Ongkir               120 non-null    object
dtypes: object(6)
memory usage: 5.8+ KB


Berdasarkan informasi yang diperoleh dari dataframe 'df':
Data hasil scraping memiliki total 120 baris (indeks 0 sampai dengan 119) dan memiliki 6 kolom.

6 kolom pada yang diperoleh dari hasil scraoing yaitu:
1. Nama Produk
2. Harga Sebelum Diskon
3. Diskon
4. Harga Setelah Diskon
5. Jumlah Terjual atau Ulasan
6. Gratis Ongkir

Setiap kolom memiliki karakteristik masing-masing, diantaranya:

1. Nama Produk
    - Berisi nama produk yang di jual pada laman website dekoruma.
    - Dari total 120 data, semuanya tersedia, artinya tidak ada missing value atau data hilang.
    - Tipe data adalah object. Tipe ini sudah sesuai karena nama produk memang berupa teks sehingga tidak perlu dilakukan perubahan.

2. Harga Sebelum Diskon
    - Berisi harga produk sebelum dipotong diskon, yang di jual pada laman website dekoruma.
    - Dari total 120 data, semuanya tersedia, artinya tidak ada missing value atau data hilang.
    - Tipe data adalah object. Tipe ini kurang sesuai karena harga produk seharusnya berupa integer agar data dapat digunakan untuk perhitungan. Oleh karena itu perlu dilakukan perubahan data dengan menghapus karakter non numeric dan mengkonversi menjadi integer.
    
    Contoh data pada kolom ini:
     
    Rp1,485,000

    Perlu diubah menjadi:
    
    1485000

    Maka pada perubahan data nantinya, perlu menghilangkan karakter 'Rp' tanda koma (,), kemudian mengubah data menjadi integer.

3. Diskon
    - Berisi diskon atau potongan harga produk di jual pada laman website dekoruma.
    - Dari total 120 data, 114 non-null, artinya terdapat missing value atau data hilang. (Hal ini kemungkinan disebabkan oleh produk-produk yang tidak memiliki diskon)
    - Tipe data adalah object. Tipe ini kurang sesuai karena diskon seharusnya berupa float agar data dapat digunakan untuk perhitungan. Oleh karena itu perlu dilakukan perubahan data dengan menghapus karakter non numeric dan mengkonversi menjadi float.
    - Perbaikan lain yang diperlukan pada kolom ini yaitu mengubah data-data 'None' menjadi 0 (integer) agar dapat digunakan dalam perhitungan dan analisis.
    
    Contoh data pada kolom ini:
     
    57%	

    Perlu diubah menjadi:
    
    0,57

    Maka pada perubahan data nantinya, perlu menghilangkan karakter '%' kemudian mengubah data menjadi integer, dan mengembalikan skalanya menjaid desimal agar sama dengan presentase awal.

4. Harga Setelah Diskon
    - Berisi harga produk setelah dipotong diskon, yang di jual pada laman website dekoruma.
    - Dari total 120 data, semuanya tersedia, artinya tidak ada missing value atau data hilang. (Hal ini bisa terjadi karena jika tidak ada diskon, maka proses scraping mengambil data yang sama dengan harga sebelum diskon, jadi dianggap diskonnya 0)
    - Tipe data adalah object. Tipe ini kurang sesuai karena harga produk seharusnya berupa integer agar data dapat digunakan untuk perhitungan. Oleh karena itu perlu dilakukan perubahan data dengan menghapus karakter non numeric dan mengkonversi menjadi integer.
    
    Contoh data pada kolom ini:
     
    Rp859,000	

    Perlu diubah menjadi:
    
    859000

    Maka pada perubahan data nantinya, perlu menghilangkan karakter 'Rp' tanda koma (,), kemudian mengubah data menjadi integer.

5. Jumlah Terjual atau Ulasan
    - Berisi jumlah terjual atau ulasan yang didapat dari produk yang di jual pada laman website dekoruma.
    - Dari total 120 data, 101 non-null, artinya terdapat missing value atau data hilang. (Hal ini kemungkinan disebabkan oleh produk-produk yang belum pernah terjual atau belum memiliki ulasan sama sekali)
    - Tipe data adalah object. Tipe ini kurang sudah sesuai karena kolom ini beberapa jumlah terjual dan beberapa berisikan ulasan (jadi tercampur). Oleh karena itu, keterangan 'terjual' atau 'ulasan' di belakang angka tidak boleh dihapus karena sebagai penjelas. 
    - Perbaikan yang diperlukan pada kolom ini yaitu menghapus tanda kurung '()' pada data karena tidak diperlukan. Selain itu, juga perlu untuk mengubah data dengan value 'None' dengan keterangan agar jelas apabila tidak ada pembelian maupun ulasan.
    
    Contoh data pada kolom ini:
     
    (4055 Terjual)	

    Perlu diubah menjadi:
    
    4055 Terjual

6. Gratis Ongkir
    - Berisi jenis gratis ongkir yang tersedia yang bisa didapat ketika membeli produk yang di jual pada laman website dekoruma.
    - Dari total 120 data, semuanya tersedia, artinya tidak ada missing value atau data hilang.
    - Tipe data  adalah object. Tipe ini sudah sesuai karena gratis ongkir memang berupa teks sehingga tidak perlu dilakukan perubahan.

## 4.2 Data Pre-Processing (Cleaning and Fixing The Data)

Seperti yang sudah dijelaskan pada bagian sebelumnya, pada bagian ini akan dilakukan perbaikan pada kolom-kolom yang perlu diperbaiki. Diantaranya: Harga Sebelum Diskon, Diskon, Harga Setelah Diskon, serta Jumlah Terjual dan Ulasan.

### 4.2.1. Memperbaiki Harga Produk Sebelum Diskon

In [190]:
df.head()

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,"Rp1,485,000",42%,"Rp859,000",(4055 Terjual),se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,"Rp1,980,000",47%,"Rp1,049,000",(3612 Terjual),se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,"Rp2,178,000",52%,"Rp1,029,000",(2661 Terjual),se-Jabodetabek
3,Heim Studio ISY Meja TV,"Rp1,434,000",39%,"Rp869,000",(3153 Terjual),se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,"Rp3,636,000",48%,"Rp1,859,000",(3135 Terjual),se-Jabodetabek


In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Nama Produk                 120 non-null    object
 1   Harga Sebelum Diskon        120 non-null    object
 2   Diskon                      114 non-null    object
 3   Harga Setelah Diskon        120 non-null    object
 4   Jumlah Terjual atau Ulasan  101 non-null    object
 5   Gratis Ongkir               120 non-null    object
dtypes: object(6)
memory usage: 5.8+ KB


Pada data harga sebelum diskon, tipe data adalah object. Tipe ini kurang sesuai karena harga produk seharusnya berupa integer agar data dapat digunakan untuk perhitungan dan analisis lebih lanjut.p  Perbaikan yang perlu dilakukan adalah menghapus karakter non numeric dan mengkonversi menjadi integer.

Contoh data pada kolom ini:
     
Rp1,485,000

Perlu diubah menjadi:
    
1485000

Maka pada perubahan data nantinya, perlu menghilangkan karakter 'Rp' tanda koma (,), kemudian mengubah data menjadi integer.

In [192]:
df["Harga Sebelum Diskon"] = df["Harga Sebelum Diskon"].str.replace("Rp", "") # Menghapus "Rp" pada harga sebelum diskon
df["Harga Sebelum Diskon"] = df["Harga Sebelum Diskon"].str.replace(",", "") # Menghapus "," pada harga sebelum diskon
df["Harga Sebelum Diskon"] = df["Harga Sebelum Diskon"].astype("Int64") # Mengubah tipe data menjadi integer

In [193]:
df.head()

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,1485000,42%,"Rp859,000",(4055 Terjual),se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,1980000,47%,"Rp1,049,000",(3612 Terjual),se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,2178000,52%,"Rp1,029,000",(2661 Terjual),se-Jabodetabek
3,Heim Studio ISY Meja TV,1434000,39%,"Rp869,000",(3153 Terjual),se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,3636000,48%,"Rp1,859,000",(3135 Terjual),se-Jabodetabek


In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Nama Produk                 120 non-null    object
 1   Harga Sebelum Diskon        120 non-null    Int64 
 2   Diskon                      114 non-null    object
 3   Harga Setelah Diskon        120 non-null    object
 4   Jumlah Terjual atau Ulasan  101 non-null    object
 5   Gratis Ongkir               120 non-null    object
dtypes: Int64(1), object(5)
memory usage: 5.9+ KB


Berdasarkan output di atas, terlihat bahwa karakter 'Rp' tanda koma (,) pada data Harga Sebelum Diskon sudah berhasil dihilangkan. Selain itu tipe data juga sudah berhasil diubah menjadi integer. Sehingga, perbaikan untuk data Harga Sebelum Diskon sudah selesai.

### 4.2.2. Memperbaiki Diskon

In [195]:
df["Diskon"] = df["Diskon"].str.replace("%", "") # Menghapus "," pada harga sebelum diskon
df["Diskon"] = df["Diskon"].astype("float") # Mengubah tipe data menjadi float
df["Diskon"] = df["Diskon"] / 100 # Membagi data dengan 100 agar skala nya sama dengan presentase awal (supaya nilainya tidak berubah)

In [196]:
# Menangani missing value

df["Diskon"] = df["Diskon"].replace({np.NaN: 0}) # Mengganti data 'NaN' dengan nilai 0 (Karena data NaN berarti tidak ada diskon sehingga bisa diganti dengan 0)

In [197]:
df.head()

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,1485000,0.42,"Rp859,000",(4055 Terjual),se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,1980000,0.47,"Rp1,049,000",(3612 Terjual),se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,2178000,0.52,"Rp1,029,000",(2661 Terjual),se-Jabodetabek
3,Heim Studio ISY Meja TV,1434000,0.39,"Rp869,000",(3153 Terjual),se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,3636000,0.48,"Rp1,859,000",(3135 Terjual),se-Jabodetabek


In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nama Produk                 120 non-null    object 
 1   Harga Sebelum Diskon        120 non-null    Int64  
 2   Diskon                      120 non-null    float64
 3   Harga Setelah Diskon        120 non-null    object 
 4   Jumlah Terjual atau Ulasan  101 non-null    object 
 5   Gratis Ongkir               120 non-null    object 
dtypes: Int64(1), float64(1), object(4)
memory usage: 5.9+ KB


Berdasarkan output di atas, terlihat bahwa karakter '%' pada data Diskon sudah berhasil dihilangkan. Selain itu tipe data juga sudah berhasil diubah menjadi float. Data juga dibagi dengan 100 agar nilainya tidak berubah. Data yang berisikan 'Nan' juga sudah diubah menjadi 0 sehingga tidak ada lagi missing value. Sehingga, perbaikan untuk data Diskon sudah selesai.

### 4.2.3. Memperbaiki Harga Setelah Diskon

In [199]:
df["Harga Setelah Diskon"] = df["Harga Setelah Diskon"].str.replace("Rp", "") # Menghapus "Rp" pada harga setelah diskon
df["Harga Setelah Diskon"] = df["Harga Setelah Diskon"].str.replace(",", "") # Menghapus "," pada harga setelah diskon
df["Harga Setelah Diskon"] = df["Harga Setelah Diskon"].astype("Int64") # Mengubah tipe data menjadi integer

In [200]:
df.head()

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,1485000,0.42,859000,(4055 Terjual),se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,1980000,0.47,1049000,(3612 Terjual),se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,2178000,0.52,1029000,(2661 Terjual),se-Jabodetabek
3,Heim Studio ISY Meja TV,1434000,0.39,869000,(3153 Terjual),se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,3636000,0.48,1859000,(3135 Terjual),se-Jabodetabek


In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nama Produk                 120 non-null    object 
 1   Harga Sebelum Diskon        120 non-null    Int64  
 2   Diskon                      120 non-null    float64
 3   Harga Setelah Diskon        120 non-null    Int64  
 4   Jumlah Terjual atau Ulasan  101 non-null    object 
 5   Gratis Ongkir               120 non-null    object 
dtypes: Int64(2), float64(1), object(3)
memory usage: 6.0+ KB


Berdasarkan output di atas, terlihat bahwa karakter 'Rp' tanda koma (,) pada data Harga Setelah Diskon sudah berhasil dihilangkan. Selain itu tipe data juga sudah berhasil diubah menjadi integer. Sehingga, perbaikan untuk data Harga Setelah Diskon sudah selesai.

### 4.2.4. Memperbaiki Jumlah Terjual atau Ulasan

In [202]:
df["Jumlah Terjual atau Ulasan"] = df["Jumlah Terjual atau Ulasan"].str.replace("(", "") # Menghapus "(" pada harga setelah diskon
df["Jumlah Terjual atau Ulasan"] = df["Jumlah Terjual atau Ulasan"].str.replace(")", "") # Menghapus ")" pada harga setelah diskon

In [205]:
# Menangani missing value

df["Jumlah Terjual atau Ulasan"] = df["Jumlah Terjual atau Ulasan"].replace({None: "Belum Terjual atau Diulas"}) # Mengganti data 'None' dengan 'Belum Terjual atau Diulas'

In [207]:
df.head()

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,1485000,0.42,859000,4055 Terjual,se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,1980000,0.47,1049000,3612 Terjual,se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,2178000,0.52,1029000,2661 Terjual,se-Jabodetabek
3,Heim Studio ISY Meja TV,1434000,0.39,869000,3153 Terjual,se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,3636000,0.48,1859000,3135 Terjual,se-Jabodetabek


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nama Produk                 120 non-null    object 
 1   Harga Sebelum Diskon        120 non-null    Int64  
 2   Diskon                      120 non-null    float64
 3   Harga Setelah Diskon        120 non-null    Int64  
 4   Jumlah Terjual atau Ulasan  120 non-null    object 
 5   Gratis Ongkir               120 non-null    object 
dtypes: Int64(2), float64(1), object(3)
memory usage: 6.0+ KB


Berdasarkan output di atas, terlihat bahwa karakter ( dan ) pada data Jumlah Terjual atau Ulasan sudah berhasil dihilangkan. Data yang berisikan 'None' juga sudah diubah menjadi 'Belum Terjual atau Diulas' sehingga tidak ada lagi missing value. Sehingga, perbaikan untuk data Jumlah Terjual atau Ulasan sudah selesai.

## 4.3. ## Exploration Data Analysis (After Cleaning Process)

In [210]:
df.head()

,Nama Produk,Harga Sebelum Diskon,Diskon,Harga Setelah Diskon,Jumlah Terjual atau Ulasan,Gratis Ongkir
0,Heim Studio KOZU Coffee Table 2in1,1485000,0.42,859000,4055 Terjual,se-Jabodetabek
1,Heim Studio KOZU Meja Kerja,1980000,0.47,1049000,3612 Terjual,se-Jabodetabek
2,Heim Studio NANA Meja TV 2in1,2178000,0.52,1029000,2661 Terjual,se-Jabodetabek
3,Heim Studio ISY Meja TV,1434000,0.39,869000,3153 Terjual,se-Jabodetabek
4,Heim Studio YVER Meja Makan Multifungsi,3636000,0.48,1859000,3135 Terjual,se-Jabodetabek


In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nama Produk                 120 non-null    object 
 1   Harga Sebelum Diskon        120 non-null    Int64  
 2   Diskon                      120 non-null    float64
 3   Harga Setelah Diskon        120 non-null    Int64  
 4   Jumlah Terjual atau Ulasan  120 non-null    object 
 5   Gratis Ongkir               120 non-null    object 
dtypes: Int64(2), float64(1), object(3)
memory usage: 6.0+ KB


Berdasarkan informasi yang diperoleh dari output di atas, terlihat bahwa terdapat total 120 baris dan 6 kolom. Setelah dilakukan pembersihan dan perbaikan pada data-data tersebut, dari 6 kolom yang ada, semua datanya sudah lengkap (tidak ada lagi missing value). Tipe data pada setiap kolom juga sudah sesuai, yaitu:

1. Nama produk bertipe object
2. Harga Sebelum Diskon bertipe Int64  
3. Diskon bertipe float64
4. Harga Setelah Diskon bertipe Int64  
5. Jumlah Terjual atau Ulasan bertipe object 
6. Gratis Ongkir bertipe object 

Karena seluruh data sudah sesuai, maka data ini dapat digunakan untuk analisis lebih lanjut.

# 5. Menyimpan Hasil Scraping ke CSV

In [ ]:
df.to_csv('cleaned_retail_data.csv', index=False) # index = false agar indexnya tidak tersimpan

# 6. SQL

Program (query) ini dibuat untuk pengelolaan dan pengolahan data hasil scraping website dekoruma. Mulai dari pembuatan database, tabel, dan normalisasi apabila diperlukan.

## 6.1. Membuat Database

```sql
-- Query ini digunakan untuk membuat database
CREATE DATABASE ecommerce; 
--(Note: Ini seharusnya berada pada halaman query yang berbeda, tetapi saya copy di sini  agar menjadi 1 file sql)
```


## 6.2. Membuat Tabel Staging

```sql
-- Query ini digunakan untuk membuat table "staging"
CREATE TABLE staging (
		id SERIAL,
		nama_produk VARCHAR (250),
		harga_sebelum_diskon INT,
		diskon FLOAT,
		harga_setelah_diskon INT,
		jumlah_terjual_ulasan VARCHAR (250),
		gratis_ongkir VARCHAR (250)
		);

-- Query ini digunakan untuk mengambil semua data dari tabel "staging", tetapi dalam hal ini digunakan untuk memastikan apakah kolom pada table "staging" sudah sesuai
SELECT * FROM staging;
```

```sql
-- Query ini digunakan untuk memasukkan data pada tabel "staging", di mana data yang digunakan berasal dari file csv
COPY staging (nama_produk, harga_sebelum_diskon, diskon, harga_setelah_diskon, jumlah_terjual_ulasan, gratis_ongkir)
FROM 'C:\Users\cleaned_retail_data.csv' DELIMITER ',' CSV HEADER; 

-- Query ini digunakan untuk mengambil semua data dari tabel "staging"
SELECT * FROM staging;

```

## 6.3. Normalisasi

Pada kasus ini, normalisasi pada data tidak diperlukan karena seluruh kolom sudah bernilai single value (tidak ada data multiple value). 
Selain itu, data pada tabel staging tidak memiliki data duplikat. 
Setiap produk memiliki satu harga sebelum diskon, satu diskon, satu harga setelah diskon, dan satu keterangan terkait terjual atau ulasan, serta gratis ongkir yang didapatkan.
Oleh karena itu, tidak perlu dilakukan normalisasi pada data ini.


## 6.4. Menyimpan Tabel Staging

```sql
COPY staging TO 'C:/Users/staging_retail_data.csv' WITH (FORMAT csv, HEADER); 
```